***
# Nettoyage des données
***

In [1]:
# ====== Importer les library ====== #
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

In [2]:
# ====== Déterminer les path ====== #
import os
cwd = os.getcwd()
parent = os.path.dirname(cwd)
data_path = parent + '\\data\\'

In [3]:
# ====== Importer les df ====== #
comments = pd.read_csv(data_path + 'Comments.csv')
posts = pd.read_csv(data_path + 'Posts.csv')

comments.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time,id,like_count,message,parent,postId
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,2023-06-02T16:06:59+0000,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,2023-06-02T16:21:43+0000,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613


In [4]:
test = pd.read_csv(data_path + 'Test.csv')
test.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time,id,like_count,message,parent,postId
0,0.000548,0.007893,0.011800,0.000517,0.005419,0.006754,-1,2023-06-01T18:32:37+0000,721844289946959_831368741921536,0,Mon ex🤔?,NaN,192978590727638_721844289946959
1,0.125566,0.469236,0.501357,0.169603,0.012389,0.434752,-1,2023-06-01T18:51:50+0000,721844289946959_9565562033514339,0,"""Quel est ton nom ? Je te l' ordonne de le di...",NaN,192978590727638_721844289946959


In [4]:
# ====== Merge les df ====== #
df_merge = pd.merge(comments, posts, left_on='postId', right_on='id', indicator=True, suffixes=('_comments', '_posts'))
df_merge.head(2)


# Analyse du merge 
unique_list = []
for value in df_merge['_merge'].unique():
    unique_list.append(value)

if (len(unique_list) == 1) & (unique_list[0] == 'both'):
    print('Tous les lignes merge')
else :
    print('Certaines lignes ne sont pas merge')


df_merge = df_merge.drop(columns='_merge')

Tous les lignes merge


In [6]:
# Attention au Nan
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_17240\284039997.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['created_time_comments', 'id_comments', 'message_comments', 'parent',
       'postId', 'attachments.data', 'created_time_posts', 'id_posts',
       'mainTopic', 'message_posts', 'permalink_url', 'secondTopic', 'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
### Compléter les valeurs manquantes

#### Solution
- Drop parent
- Drop missings title row 
***

In [13]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time_comments,id_comments,like_count,message_comments,parent,postId,attachments.data,created_time_posts,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,2023-06-02T16:06:59+0000,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",2023-06-02T16:03:24+0000,192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,2023-06-02T16:21:43+0000,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",2023-06-02T16:03:24+0000,192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence


In [9]:
# ====== Visualisation des valeurs manquantes ====== #
print('Missing value')
print('*'*25)
print(f'Nombre de lignes : {len(df_merge)}')
for col in df_merge.columns.to_list():
    nb_na = df_merge[col].isna().sum()
    perc = round(df_merge[col].isna().sum()/len(df_merge)*100)
    print(f'{col.ljust(21)} : {str(nb_na).rjust(11)} ({perc}%)')

Missing value
*************************
Nombre de lignes : 935698
IDENTITY_ATTACK       :           0 (0%)
INSULT                :           0 (0%)
PROFANITY             :           0 (0%)
SEVERE_TOXICITY       :           0 (0%)
THREAT                :           0 (0%)
TOXICITY              :           0 (0%)
comment_count         :           0 (0%)
created_time_comments :           0 (0%)
id_comments           :           0 (0%)
like_count            :           0 (0%)
message_comments      :           0 (0%)
parent                :      459661 (49%)
postId                :           0 (0%)
attachments.data      :          35 (0%)
created_time_posts    :           0 (0%)
id_posts              :           0 (0%)
mainTopic             :        8481 (1%)
message_posts         :       10158 (1%)
permalink_url         :           0 (0%)
secondTopic           :       67076 (7%)
shares                :           0 (0%)
title                 :        8457 (1%)


In [14]:
# ====== convertir le temps en format datetime ====== #
df_merge['comment_time'] = pd.to_datetime(df_merge['created_time_comments'])
df_merge['post_time'] = pd.to_datetime(df_merge['created_time_posts'])
df_merge = df_merge.drop(columns=['created_time_comments', 'created_time_posts'])
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,id_comments,like_count,message_comments,parent,postId,attachments.data,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title,comment_time,post_time
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:06:59+00:00,2023-06-02 16:03:24+00:00
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:21:43+00:00,2023-06-02 16:03:24+00:00


In [15]:
# ====== Éliminer les colonnes non-pertinantes ====== #
colones_non_utiles = ['id_comments', 'parent', 'postId', 'attachments.data', 'id_posts', 'permalink_url']
df_merge = df_merge.drop(columns=colones_non_utiles)
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,comment_time,post_time
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:06:59+00:00,2023-06-02 16:03:24+00:00
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:21:43+00:00,2023-06-02 16:03:24+00:00


In [32]:
# Attribut différence de temps
df_merge['time_difference'] = df_merge['comment_time'] - df_merge['post_time'] 
df_merge['time_difference'] = df_merge['time_difference'].astype('timedelta64[m]')
df_merge.head(2)

# Attributs temps solo
df_merge['year'] = df_merge['comment_time'].dt.year
df_merge['month'] = df_merge['comment_time'].dt.month
df_merge['weekday'] = df_merge['comment_time'].dt.weekday       # Monday=0, Sunday=6
df_merge = df_merge.drop(columns=['comment_time', 'post_time'])
df_merge.head(2)


,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [34]:
# Attention au Nan
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_clean.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_17240\3880442555.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_clean.h5', key='s')


***
## Les sujets : Étiquettage
***

In [27]:
df_merge[["title"]]

,title
0,Les incohérences de l’ingérence
1,Legault veut empêcher les antiavortement de fa...
2,S’envoyer en l’air à 97 ans
3,Une famille québécoise vit dans une géonef aut...
4,Influenceurs : gare aux publicités trompeuses
...,...
475371,La CAQ prévoit améliorer les conditions de tra...
475372,La CAQ prévoit améliorer les conditions de tra...
475373,La CAQ prévoit améliorer les conditions de tra...
475374,La CAQ prévoit améliorer les conditions de tra...


***
## Extraction des mots dans les publications
***

1. Mots pertinent (les titres et les commentaires, les commentaires parents)
2. Les émoji

In [4]:
df_clean =pd.read_hdf(cwd + '\\dataframe\\' + 'df_clean.h5')
df_clean.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [5]:
df_clean = df_clean.head(100).copy()

In [6]:
import spacy
nlp = spacy.load('fr_core_news_sm')

def cleanTextWithSpacy(text):
    doc = nlp(text)
    filtered_words = [token.text for token in doc if (not token.is_stop) & (not token.is_punct)]
    clean_text = ' '.join(filtered_words).lower()
    return clean_text

df_clean['messages_clean'] = df_clean['message_comments'].apply(lambda x: cleanTextWithSpacy(x))
df_clean.head()

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,messages_clean
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,faux jamais temps réélu parti crève faim jamai...
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,dieu prend sérieusement idiots 😅
2,0.029144,0.269578,0.016753,0.006104,0.005864,0.246033,0,0,"Denise Morais ..Oui et avec raison ! (Car ""nou...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,86.0,2023,6,4,denise morais oui raison majorité tyrannique...
3,0.036105,0.402434,0.479813,0.023411,0.008220,0.372227,0,0,Gros show. Mais pendant la plandémie chinoise ...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,94.0,2023,6,4,gros show plandémie chinoise tlm fermait bien ...
4,0.280349,0.520819,0.163573,0.169603,0.008790,0.439655,0,0,On apellait ça chinada pis les vaxxinés compre...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,95.0,2023,6,4,apellait chinada pis vaxxinés comprenaient rie...


In [9]:
text = df_clean['messages_clean'][1]
text


'dieu prend sérieusement idiots 😅'

In [21]:
import re
import emoji

def findEmoji(text):
    emojis = re.findall(r'[\U0001F300-\U0001F5FF|\U0001F600-\U0001F64F|\U0001F680-\U0001F6FF|\u2600-\u26FF\u2700-\u27BF]', text)
    return emojis

##### Créer une liste de valeur plutot qu'un bloc de text #####
def emojiToText(text):
    emojiInText = emoji.demojize(text)
    return emojiInText

df_clean['emojis'] = df_clean['messages_clean'].apply(lambda x: findEmoji(x))
df_clean['emojis_text'] = df_clean['emojis'].apply(lambda x: emojiToText(x))
df_clean.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,messages_clean,emojis,emojis_text
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,faux jamais temps réélu parti crève faim jamai...,[],
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,dieu prend sérieusement idiots 😅,[😅],:grinning_face_with_sweat:


In [33]:
text = df_clean['emojis'][0]

if (len(text)==0):
    print('yes')


yess


In [36]:
df_clean[df_clean['emojis_text'].str.len() > 0]

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,messages_clean,emojis,emojis_text
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,dieu prend sérieusement idiots 😅,[😅],:grinning_face_with_sweat:
7,0.004884,0.136078,0.015523,0.003281,0.005893,0.135618,0,2,construit on la copie du marché Champlain !!!?...,actualites,Poussière et marteaux-piqueurs accompagnent l'...,2023,0,"Le Petit Champlain en chantier, des commerçant...",130.0,2023,6,4,construit copie marché champlain bonbon électo...,[🕳],:hole:
28,0.000391,0.008653,0.009127,0.000331,0.005277,0.005686,0,0,"Ça arrive aussi à nous,comme ds le passé aux a...",sports,« C’est frustrant et fâchant de finir comme ça...,2023,0,Une erreur et une balle passée coûtent le matc...,6.0,2023,6,4,arrive ds passé équipes!!! ⚾ ️ ⚾ ️ ⚾ ️ ⚾ ️,"[⚾, ⚾, ⚾, ⚾]",:baseball::baseball::baseball::baseball:
46,0.000777,0.029824,0.022901,0.001945,0.005484,0.041173,0,0,Emilie Bolduc calisssss😡 j’aurais dû aller me ...,actualites,Le compte à rebours est amorcé pour les jeunes...,2023,15,Compte à rebours pour les moins de 14 ans au t...,37.0,2023,6,3,emilie bolduc calisssss 😡 aurais dû aller couc...,[😡],:enraged_face:
72,0.003737,0.063375,0.015523,0.001564,0.005319,0.068171,0,0,Ont fait dure ici c’est la province des lois o...,actualites,Le compte à rebours est amorcé pour les jeunes...,2023,15,Compte à rebours pour les moins de 14 ans au t...,1018.0,2023,6,4,dure ici province lois lois rien 🤦 🏻‍ ♂ ️ 🤬,"[🏻, ♂]",:light_skin_tone::male_sign:
82,0.002756,0.007798,0.018734,0.002003,0.006952,0.013194,0,0,❤️❤️❤️❤️❤️,vivre,La Piazzetta fête sa 34e année avec l’ouvertur...,dans-l-assiette,23,Un neuvième restaurant Piazzetta à Québec,336.0,2023,6,4,❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️,"[❤, ❤, ❤, ❤, ❤]",:red_heart::red_heart::red_heart::red_heart::r...
83,0.001434,0.011579,0.102401,0.004253,0.005845,0.030742,0,1,"🍕 Oh wow, j’arrive !",vivre,La Piazzetta fête sa 34e année avec l’ouvertur...,dans-l-assiette,23,Un neuvième restaurant Piazzetta à Québec,352.0,2023,6,4,🍕 oh wow arrive,[🍕],:pizza:
93,0.000629,0.029053,0.008768,0.000505,0.005299,0.033806,0,0,Martin Boucher pourquoi as-tu peur d’un référe...,actualites,"Le chef de l'opposition officielle, Claude Vil...",actualites-locales,0,Villeneuve veut « l’heure juste » sur le calen...,474.0,2023,6,4,martin boucher peur référendum bien temps pass...,"[😉, 😉, 😉]",:winking_face::winking_face::winking_face:
